# Testing DWT.py

## Parameters

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import math
import numpy as np
from scipy import signal
import cv2
import os
import pywt
import pylab
!ln -sf ~/quantization/deadzone_quantizer.py .
import image_1
import image_3
import DWT

In [ ]:
#prefix = "/home/vruiz/MRVC/sequences/stockholm/"
prefix = "/home/vruiz/MRVC/sequences/lena_color/"
frame = image_3.read(prefix, 0)

In [ ]:
image_3.show(frame, prefix)

## Testing reversebility and structure of a decomposition
In the RGB domain.

In [ ]:
color_decomposition = DWT.analyze(frame, N_levels=3)
reconstructed_frame = DWT.synthesize(color_decomposition, N_levels=3).astype(np.int16)
(frame == reconstructed_frame).all()

In [ ]:
type(color_decomposition)

In [ ]:
len(color_decomposition)

In [ ]:
type(color_decomposition[0])

In [ ]:
color_decomposition[0].shape

In [ ]:
color_decomposition[0].dtype

In [ ]:
type(color_decomposition[1])

In [ ]:
len(color_decomposition[1])

In [ ]:
lowest_resolution = color_decomposition[0]
print(type(lowest_resolution), lowest_resolution.shape)
for resolution in color_decomposition[1:]:
    print(type(resolution))
    for subband in resolution:
        print(type(subband), subband.shape)

In [ ]:
reconstructed_frame.shape

In [ ]:
image_3.show(frame, "original")

In [ ]:
image_3.show(reconstructed_frame, "reconstruction")

In [ ]:
image_3.show(image_3.normalize(frame - reconstructed_frame), "difference")

### Showing (glued) decompositions

In [ ]:
def _extract_decomposition(color_decomposition, component_I):
    '''Extract a component (in form of a decomposition) from a color
decomposition.

    Parameters
    ----------
    color_decomposition : Python-list
        An input list of color SRLs.
    component_index : int
        The component to extract.

    Returns
    -------
    A (monochromatic) decomposition: list

    '''
    decomposition = [color_decomposition[0][..., component_I]]
    for color_resolution in color_decomposition[1:]:
        resolution = [] 
        for color_subband in color_resolution:
            resolution.append(color_subband[..., component_I])
        decomposition.append(tuple(resolution))
    return decomposition

def _glue_decomposition(decomposition):
    '''Convert a list of (monocromatic) subbands to a (row, column) NumPy array.

    Parameters
    ----------
    decomposition : Python-list
        The input decomposition to convert in a np.ndarray.

    Returns
    -------
    The glued decomposition : a (row, column) np.ndarray.
        A single monochromatic image with all the wavelet coefficients.
    The generated slices : a Python-list.
        The data structure in the "wavedec2" format that describes the original decomposition.
    '''
    glued_decomposition, slices = pywt.coeffs_to_array(decomposition)
    return glued_decomposition, slices

def _glue_color_decomposition(color_decomposition):
    '''Convert a list of color SRLs to a (row, column, component) NumPy array.

    Parameters
    ----------
    color_decomposition : Python-list
        The input decomposition to convert in a np.ndarray.

    Returns
    -------
    The glued color decomposition : a [row, column, component] np.ndarray.
        A single color image with all the wavelet coefficients.
    The list of the generated slices : a Python-list (with one item per component).
        The description of the data structure in the "wavedec2" format that describes the original decomposition.
    '''
    N_comps = color_decomposition[0].shape[2]
    glued_decompositions = []
    slices = [None]*3
    for component_I in range(N_comps):
        decomposition  = extract_decomposition(color_decomposition, component_I)
        glued_decomposition, slices[component_I] = glue_decomposition(decomposition)
        glued_decompositions.append(glued_decomposition)
    N_rows = glued_decompositions[0].shape[0]
    N_cols = glued_decompositions[0].shape[1]
    glued_color_decomposition = np.empty(shape=(N_rows, N_cols, N_comps), dtype=np.float64)
    for component_I in range(N_comps):
        glued_color_decomposition[..., component_I] = glued_decompositions[component_I][:]
    return glued_color_decomposition, slices

glued_color_decomposition = DWT.glue_color_decomposition(color_decomposition)[0]

In [ ]:
image_3.show(image_3.normalize(glued_color_decomposition))

### Reading and writting decompositions

In [ ]:
def add(decomposition, val=32768, dtype=np.uint16):
    new_decomp = [(decomposition[0] + val).astype(dtype)]
    for resolution in decomposition[1:]:
        new_resol = []
        for subband in resolution:
            new_resol.append((subband + val).astype(dtype))
        new_decomp.append(tuple(new_resol))
    return new_decomp

def set_type(decomposition, dtype):
    new_decomp = [decomposition[0].astype(dtype)]
    for resolution in decomposition[1:]:
        new_resol = []
        for subband in resolution:
            new_resol.append(subband.astype(dtype))
        new_decomp.append(tuple(new_resol))
    return new_decomp

def copy(decomposition):
    new_decomp = [decomposition[0].copy()]
    for resolution in decomposition[1:]:
        new_resol = []
        for subband in resolution:
            new_resol.append(subband.copy())
            

In [ ]:
DWT.write_decomposition(DWT.add(color_decomposition, 32768, np.uint16), "/tmp/", 0, N_levels=3)

In [ ]:
_N_levels = 5
def read_decomposition(prefix, image_number=0, N_levels=_N_levels):
    '''Read a color decomposition from the disk (one file per color subband).

    Parameters
    ----------
    prefix : A Python-string.
        The prefix of the input files.
    image_number : A signed integer.
        The image number in a possible sequence of images (frames).

    Returns
    -------
    color_decomposition : A Python-list of color SRLs.
        The color decomposition read from the disk.

    '''
    #LL = L.read(f"{prefix}_{N_levels+1}", image_number)
    LL = image_3.read(f"{prefix}LL{N_levels}", image_number)
    color_decomposition = [LL]
    resolution_I = N_levels
    for l in range(N_levels, 0, -1):
        subband_names = ["LH", "HL", "HH"]
        sb = 0
        resolution = []
        for sbn in subband_names:
            resolution.append(image_3.read(f"{prefix}{sbn}{resolution_I}", image_number))
            sb += 1
        color_decomposition.append(tuple(resolution))
        resolution_I -= 1    

_color_decomposition = DWT.read_decomposition("/tmp/", 0, N_levels=3)

### Reading and writing glued decompositions

In [ ]:
def extract_decomposition(color_decomposition, component_I):
    '''Extract a component (in form of a decomposition) from a color
decomposition.

    Parameters
    ----------
    color_decomposition : Python-list
        An input list of color SRLs.
    component_index : int
        The component to extract.

    Returns
    -------
    A (monochromatic) decomposition: list

    '''
    decomposition = [color_decomposition[0][..., component_I]]
    for color_resolution in color_decomposition[1:]:
        resolution = [] 
        for color_subband in color_resolution:
            resolution.append(color_subband[..., component_I])
        decomposition.append(tuple(resolution))
    return decomposition

def glue_decomposition(decomposition):
    '''Convert a list of (monocromatic) subbands to a (row, column) NumPy array.

    Parameters
    ----------
    decomposition : Python-list
        The input decomposition to convert in a np.ndarray.

    Returns
    -------
    The glued decomposition : a (row, column) np.ndarray.
        A single monochromatic image with all the wavelet coefficients.
    The generated slices : a Python-list.
        The data structure in the "wavedec2" format that describes the original decomposition.
    '''
    glued_decomposition, slices = pywt.coeffs_to_array(decomposition)
    return glued_decomposition, slices

def glue_color_decomposition(color_decomposition):
    '''Convert a list of color SRLs to a (row, column, component) NumPy array.

    Parameters
    ----------
    color_decomposition : Python-list
        The input decomposition to convert in a np.ndarray.

    Returns
    -------
    The glued color decomposition : a [row, column, component] np.ndarray.
        A single color image with all the wavelet coefficients.
    The list of the generated slices : a Python-list (with one item per component).
        The description of the data structure in the "wavedec2" format that describes the original decomposition.
    '''
    N_comps = color_decomposition[0].shape[2]
    dtype = color_decomposition[0].dtype
    glued_decompositions = []
    slices = [None]*3
    for component_I in range(N_comps):
        decomposition  = extract_decomposition(color_decomposition, component_I)
        glued_decomposition, slices[component_I] = glue_decomposition(decomposition)
        glued_decompositions.append(glued_decomposition)
    N_rows = glued_decompositions[0].shape[0]
    N_cols = glued_decompositions[0].shape[1]
    glued_color_decomposition = np.empty(shape=(N_rows, N_cols, N_comps), dtype=dtype)
    for component_I in range(N_comps):
        glued_color_decomposition[..., component_I] = glued_decompositions[component_I][:]
    return glued_color_decomposition, slices

def write(color_decomposition, prefix=str, image_number=0):
    '''Write a color decomposition into disk file, as a single color
image, in glued format.

    Parameters
    ----------
    color_decomposition : A Python-list of color SRLs.
        The color decomposition to write in disk.
    prefix : A Python-string.
        The prefix of the output file.
    image_number : A signed integer.
        The image number in a possible sequence of images (frames).

    Returns
    -------
    The list of slices that describes the structure of the decomposition of each component.

    '''
    glued_color_decomposition, slices = glue_color_decomposition(color_decomposition)
    output_length = image_3.write(glued_color_decomposition, prefix, image_number)
    return output_length, slices

N_bytes, slices = DWT.write(DWT.add(color_decomposition, 32768, np.uint16), "/tmp/", 0)

In [ ]:
def unglue_decomposition(glued_decomposition, slices):
    '''Convert a glued decomposition (a (row, column) np.array) in a list of tuples
of subbands (each one a (row, column) np.ndarray).
    Parameters
    ----------
    glued_decomposition : np.ndarray
        The glued decomposition to split.
    slices : list
        The structure of the decomposition in "wavdec2" format.
    Returns
    -------
    The decomposition : a Python-list of SRLs.
    '''
    decomposition = pywt.array_to_coeffs(glued_decomposition, coeff_slices=slices, output_format='wavedec2')
    return decomposition

def unglue_color_decomposition(glued_color_decomposition, slices):
    '''Convert a (row, column, component) NumPy array in a list of color
SRLs.

    Parameters
    ----------
    glued_color_decomposition : a (row, column, component) NumPy array.
        The glued color decomposition to split.
    slices : list
        The description of the structures (one per component)  of the decomposition in "wavdec2" format.

    Returns
    -------
    The color decomposition : a Python-list of SRLs.
       The same structure as analyze().
    '''

    N_levels = len(slices)
    N_comps = glued_color_decomposition.shape[2]
    decompositions = []
    for component_I in range(N_comps):
        decompositions.append(unglue_decomposition(glued_color_decomposition[..., component_I], slices[component_I]))
    # "decompositions" is a list with three decompositions.

    ########### notice that the following code is used also un analyze() ##############
    color_decomposition = []
    # LL^N_levels and H^N_levels subbands (both have the same resolution)
    N_rows_subband, N_columns_subband = decompositions[0][0].shape # All subbands in the SRL with the same shape
    LL = np.empty(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
    LH = np.zeros(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
    HL = np.zeros(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
    HH = np.zeros(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
    for component_I in range(N_comps):
        LL[:,:, component_I] = decompositions[component_I][0][:,:]
        LH[:,:, component_I] = decompositions[component_I][1][0][:,:]
        HL[:,:, component_I] = decompositions[component_I][1][1][:,:]
        HH[:,:, component_I] = decompositions[component_I][1][2][:,:]
    color_decomposition.append(LL)
    color_decomposition.append((LH, HL, HH))
    
    # For the rest of SRLs (have increasing resolutions)
    for resolution_I in range(2, N_levels+1):
        N_rows_subband, N_columns_subband = decompositions[0][resolution_I][0].shape
        prev_N_columns_subband = N_columns_subband
        LH = np.zeros(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
        HL = np.zeros(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
        HH = np.zeros(shape=(N_rows_subband, N_columns_subband, N_comps), dtype=np.float64)
        for component_I in range(N_comps):
            LH[:,:, component_I] = decompositions[component_I][resolution_I][0][:,:]
            HL[:,:, component_I] = decompositions[component_I][resolution_I][1][:,:]
            HH[:,:, component_I] = decompositions[component_I][resolution_I][2][:,:]
        color_decomposition.append((LH, HL, HH))
    return color_decomposition

def read(slices, prefix, image_number=0):
    '''Read a color decomposition from a disk file.

    Parameters
    ----------
    slices : a Python-list
        The structure of the decomposition of each component.
    prefid: A Python-string
        The prefix of the inputf¡ file.
    image_number : A signed integer.
        The image number in a possible sequence of images (frames).

    Returns
    -------
    A color decomposition : a Python-list of color subbands.
    '''
    glued_color_decomposition = image_3.read(prefix, image_number)
    color_decomposition = unglue_color_decomposition(glued_color_decomposition, slices)
    return color_decomposition

_color_decomposition = DWT.read(slices, "/tmp/", 0)

In [ ]:
glued_color_decomposition = DWT.glue_color_decomposition(_color_decomposition)[0]
image_3.show(image_3.normalize(glued_color_decomposition))

### In the YCoCg domain

In [ ]:
YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
YCoCg_decomposition = DWT.analyze(YCoCg_frame, N_levels=2)
YCoCg_reconstructed_frame = DWT.synthesize(YCoCg_decomposition, N_levels=2)
RGB_reconstructed_frame = YCoCg.to_RGB(YCoCg_reconstructed_frame).astype(np.int16)
(RGB_frame == RGB_reconstructed_frame).all()

In [ ]:
image.show(RGB_frame, "original") 

In [ ]:
image.show(RGB_reconstructed_frame, "reconstruction")

In [ ]:
show(RGB_frame - RGB_reconstructed_frame, "difference")

### Reading and writting decompositions

In [ ]:
DWT.write(RGB_decomposition, "/tmp/", 0, N_levels=2)

In [ ]:
RGB_decomposition = DWT.read("/tmp/", 0, N_levels=2)

In [ ]:
YCoCg_reconstructed_frame = DWT.synthesize(YCoCg_decomposition, N_levels=2)
RGB_reconstructed_frame = YCoCg.to_RGB(YCoCg_reconstructed_frame).astype(np.int16)
image.show(RGB_reconstructed_frame, "reconstruction")

## Quantizing the RGB domain

In [ ]:
RGB_points = []
with open('../05-RGB_compression/RGB.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RGB_points.append((float(rate), float(_distortion)))

## Quantizing the YCoCg domain

In [ ]:
YCoCg_points = []
with open('../06-YUV_compression/YCoCg.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        YCoCg_points.append((float(rate), float(_distortion)))

## Quantizing the YCoCg+DCT domain

In [ ]:
DCT_points = []
with open('../07-DCT/DCT.txt', 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        DCT_points.append((float(rate), float(_distortion)))

## Quantizing the YCoCg/DWT domain

In [ ]:
def bytes_per_frame(img):
    frame.write(img, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bits_per_pixel(img):
    return 8*bytes_per_frame(img)/np.size(img)

def bytes_per_grayframe(img):
    cv2.imwrite("/tmp/frame.png", img)
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bits_per_graypixel(img):
    return 8*bytes_per_grayframe(img)/np.size(img)

In [ ]:
N_LEVELS = 3
def _DWT_RD_curve(RGB_frame, n_levels=N_LEVELS):
    n_channels = RGB_frame.shape[2]
    RD_points = []
    for q_step in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        YCoCg_decomposition = DWT.analyze(YCoCg_frame, n_levels=n_levels)
        dequantized_YCoCg_decomposition = []
        rate = 0
        for channel in range(n_channels):
            # In a channel there is a decomposition
            decomposition = YCoCg_decomposition[channel]
            cAn = decomposition[0]
            k, dequantized_cAn = q_deq(cAn, 1<<q_step)
            k = k.astype(np.uint8)
            dequantized_decomposition = [dequantized_cAn]
            rate += bytes_per_frame(k)
            rest_of_resolutions = decomposition[1:]
            for resolution in rest_of_resolutions:
                # In a resolution there is/are one/three subbands
                dequantized_resolution = []
                for subband in resolution:
                    k, dequantized_subband = q_deq(subband, 1<<q_step)
                    k = k.astype(np.uint8)
                    rate += bytes_per_grayframe(k)
                    dequantized_resolution.append(dequantized_subband)
                dequantized_decomposition.append(tuple(dequantized_resolution))
            dequantized_YCoCg_decomposition.append(dequantized_decomposition)
        reconstructed_YCoCg_frame = DWT.synthesize(dequantized_YCoCg_decomposition)
        reconstructed_RGB_frame = YCoCg.to_RGB(reconstructed_YCoCg_frame)
        distortion = MSE(RGB_frame, reconstructed_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
        RD_points.append((rate, distortion))
    return RD_points

def DWT_RD_curve(RGB_frame, N_levels):
    n_channels = RGB_frame.shape[2]
    RD_points = []
    for q_step in range(0, 8):
        YUV_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        YUV_decomposition = DWT.analyze(YUV_frame, N_levels=N_levels)
        dequantized_YUV_decomposition = []
        rate = 0
        cAn = YUV_decomposition[0]
        dequantized_cAn, k = q_deq(cAn, 1<<q_step)
        #rate += bytes_per_frame(k.astype(np.uint8))
        rate += image.write((k + 128).astype(np.uint8), f"/tmp/{q_step}_", 0)
        dequantized_YUV_decomposition.append(dequantized_cAn)
        rest_of_resolutions = YUV_decomposition[1:]
        for resolution in rest_of_resolutions:
            dequantized_resolution = []
            for subband in resolution:
                dequantized_subband, k  = q_deq(subband, 1<<q_step)
                #rate += bytes_per_frame(k.astype(np.uint8))
                rate += image.write((k + 128).astype(np.uint8), f"/tmp/{q_step}_", 0)
                dequantized_resolution.append(dequantized_subband)
            dequantized_YUV_decomposition.append(tuple(dequantized_resolution))
        reconstructed_YUV_frame = DWT.synthesize(dequantized_YUV_decomposition, N_levels=N_levels)
        reconstructed_RGB_frame = YCoCg.to_RGB(reconstructed_YUV_frame)
        _distortion = distortion.MSE(RGB_frame, reconstructed_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
        RD_points.append((8*rate/RGB_frame.size, _distortion))
    return RD_points

DWT_points_1 = DWT_RD_curve(RGB_frame, N_levels=1)
DWT_points_2 = DWT_RD_curve(RGB_frame, N_levels=2)
DWT_points_3 = DWT_RD_curve(RGB_frame, N_levels=3)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DWT_points_1), c='r', marker="x", label='DWT(n_levels=1)')
pylab.plot(*zip(*DWT_points_2), c='g', marker="x", label='DWT(n_levels=2)')
pylab.plot(*zip(*DWT_points_3), c='b', marker="x", label='DWT(n_levels=3)')
pylab.title("Impact of the Number of Levels")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
pylab.figure(dpi=150)
#pylab.plot(*zip(*RGB_points), c='b', marker="x", label='RGB')
#pylab.plot(*zip(*YCoCg_points), c='g', marker="x", label='YCoCg')
pylab.plot(*zip(*DWT_points_3), c='r', marker="x", label='YCoCg+DWT')
pylab.plot(*zip(*DCT_points), c='m', marker="x", label='DCT')
pylab.title("Global Performance")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
#plt.yscale('log')
pylab.show()

In [ ]:
with open('DWT3.txt', 'w') as f:
    for item in DWT_points_3:
        f.write(f"{item[0]}\t{item[1]}\n")

## Ignore the rest

In [ ]:
def RGB_to_YCoCg(RGB_frame):
    R, G, B = RGB_frame[:,:,0], RGB_frame[:,:,1], RGB_frame[:,:,2]
    YCoCg_frame = np.empty_like(RGB_frame)
    YCoCg_frame[:,:,0] =  R/4 + G/2 + B/4 
    YCoCg_frame[:,:,1] =  R/2       - B/2
    YCoCg_frame[:,:,2] = -R/4 + G/2 - B/4
    return YCoCg_frame

def YCoCg_to_RGB(YCoCg_frame):
    Y, Co, Cg = YCoCg_frame[:,:,0], YCoCg_frame[:,:,1], YCoCg_frame[:,:,2]
    RGB_frame = np.empty_like(YCoCg_frame)
    RGB_frame[:,:,0] = Y + Co - Cg 
    RGB_frame[:,:,1] = Y      + Cg
    RGB_frame[:,:,2] = Y - Co - Cg
    return RGB_frame

In [ ]:
def _average_energy(x):
    return np.sum(x.astype(np.double)*x.astype(np.double))/(np.size(x))

def _MSE(x, y):
    error_signal = x.astype(np.float32) - y
    return average_energy(error_signal)

def _RMSE(x, y):
    error_signal = x.astype(np.float32) - y
    return math.sqrt(MSE(error_signal))

In [ ]:
def bytes_per_frame(_frame):
    frame.write(_frame, "/tmp/frame")
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bits_per_pixel(img):
    return 8*bytes_per_frame(img)/np.size(img)

def bytes_per_grayframe(_frame):
    cv2.imwrite("/tmp/frame.png", _frame)
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

def bits_per_graypixel(img):
    return 8*bytes_per_grayframe(img)/np.size(img)

In [ ]:
WAVELET = pywt.Wavelet("db5")
#WAVELET = pywt.Wavelet("bior3.5")
N_LEVELS = 3

def color_DWT_analyze(color_frame, wavelet=WAVELET, n_levels=N_LEVELS):
    n_channels = color_frame.shape[2]
    color_decomposition = [None]*n_channels
    for c in range(n_channels):
        color_decomposition[c] = pywt.wavedec2(data=color_frame[:,:,c], wavelet=wavelet, mode='per', level=n_levels)
    return color_decomposition # A list of "gray" decompositions

def _color_DWT_analyze(color_frame, wavelet=WAVELET, n_levels=N_LEVELS):
    n_channels = color_frame.shape[0]
    color_decomposition = [None]*n_channels
    for c in range(n_channels):
        color_decomposition[c] = pywt.wavedec2(data=color_frame[c], wavelet=wavelet, mode='per', level=n_levels)
    return color_decomposition # A list of "gray" decompositions

def color_DWT_synthesize(color_decomposition, wavelet=WAVELET):
    n_channels = len(color_decomposition)
    #n_levels = len(color_decomposition[0])-1
    # color_decomposition[0] <- First channel
    # color_decomposition[0][0] <- cAn (lowest frequecy subband) of the first channel
    # color_decomposition[0][1] <- (cHn, cVn, cDn) (lowest high-frequency subbands) of the first channel
    # color_decomposition[0][1][0] <- cHn (LH subband) of the first channel
    # See https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2
    _color_frame = []
    for c in range(n_channels):
        frame = pywt.waverec2(color_decomposition[c], wavelet=wavelet, mode='per')
        _color_frame.append(frame)
    n_rows = _color_frame[0].shape[0]
    n_columns = _color_frame[0].shape[1]
    color_frame = np.ndarray((n_rows, n_columns, n_channels), np.float64)
    for c in range(n_channels):
        color_frame[:,:,c] = _color_frame[c][:,:]
    return color_frame

def _color_DWT_synthesize(color_decomposition, wavelet=WAVELET):
    n_channels = len(color_decomposition)
    #n_levels = len(color_decomposition[0])-1
    # color_decomposition[0] <- First channel
    # color_decomposition[0][0] <- cAn (lowest frequecy subband) of the first channel
    # color_decomposition[0][1] <- (cHn, cVn, cDn) (lowest high-frequency subbands) of the first channel
    # color_decomposition[0][1][0] <- cHn (LH subband) of the first channel
    # See https://pywavelets.readthedocs.io/en/latest/ref/2d-dwt-and-idwt.html#d-multilevel-decomposition-using-wavedec2
    _color_frame = []
    for c in range(n_channels):
        _frame = pywt.waverec2(color_decomposition[c], wavelet=wavelet, mode='per')
        _color_frame.append(_frame)
    #n_rows = _color_frame[1].shape[0]
    #n_columns = _color_frame[2].shape[1]
    #color_frame = np.ndarray((n_channels, n_rows, n_columns), np.float64)
    #for c in range(n_channels):
    #    color_frame[c] = _color_frame[c][:,:]
    return np.array(_color_frame)